Connect to Snowflake

In [ ]:
from snowflake.snowpark import Session
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions

session = Session.builder.configs(SnowflakeLoginOptions("test_conn")).create()
print(session.sql('select current_warehouse(), current_database(), current_schema()').collect())

Save model in internal named stage

In [ ]:
def save_model(session, model, stage_name, stage_path, model_file):
    import io, joblib

    input_stream = io.BytesIO()
    input_stream.name = model_file
    joblib.dump(model, input_stream)

    model_path = f'{stage_name}/{stage_path}/{model_file}'
    session.file.put_stream(input_stream, model_path, overwrite=True)
    return model_path

Read model from stage

In [ ]:
session.clear_imports()
session.clear_packages()

import cachetools
@cachetools.cached(cache={})
def read_model(model_file):
    import sys, os, joblib
    
    import_dir = sys._xoptions.get("snowflake_import_directory")
    if import_dir:
        with open(os.path.join(import_dir, model_file), 'rb') as file:
            m = joblib.load(file)
            return m